In [8]:
import requests
import defs
import pandas as pd

In [9]:
session = requests.Session()

In [10]:
ins_df = pd.read_pickle('instruments_df.pkl')

In [11]:
our_curr = ['EUR', 'USD', 'GPB', 'JPY', 'CHF', 'NZD', 'CAD']

In [12]:
def fetch_candles(pair_name, count, granularity):
    url = f"{defs.OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )
    response = session.get(url, params=params, headers=defs.SECURE_HEADER)
    return response.status_code, response.json()

In [13]:
def get_candles_df(json_response):
    prices = ['bid', 'mid', 'ask']
    ohlc = ['o', 'h', 'l', 'c']
    
    our_data = []

    for candle in json_response['candles']:
        if candle['complete'] == False:
            continue
        new_obj = {}
        new_obj['time'] = candle['time']
        new_obj['volume'] = candle['volume']
        for price in prices:
            for point in ohlc:
                new_obj[f'{price}_{point}'] = candle[price][point]
        our_data.append(new_obj)
    return pd.DataFrame.from_dict(our_data)

In [14]:
code, res = fetch_candles("EUR_USD", 10, "H1")

In [15]:
df = get_candles_df(res)

In [16]:
df.head()

,time,volume,bid_o,bid_h,bid_l,bid_c,mid_o,mid_h,mid_l,mid_c,ask_o,ask_h,ask_l,ask_c
0,2023-05-08T05:00:00.000000000Z,2083,1.10429,1.10437,1.10359,1.10381,1.10436,1.10445,1.10368,1.10388,1.10444,1.10453,1.10375,1.10396
1,2023-05-08T06:00:00.000000000Z,2622,1.10383,1.10500,1.10367,1.10477,1.10390,1.10506,1.10374,1.10484,1.10398,1.10513,1.10382,1.10490
2,2023-05-08T07:00:00.000000000Z,3596,1.10475,1.10494,1.10377,1.10487,1.10482,1.10501,1.10384,1.10494,1.10488,1.10508,1.10391,1.10501
3,2023-05-08T08:00:00.000000000Z,3920,1.10486,1.10530,1.10352,1.10355,1.10493,1.10538,1.10360,1.10362,1.10500,1.10546,1.10367,1.10370
4,2023-05-08T09:00:00.000000000Z,3674,1.10353,1.10386,1.10220,1.10352,1.10361,1.10392,1.10228,1.10360,1.10369,1.10399,1.10236,1.10368


In [17]:
def save_file(candles_df, pair, granularity):
    candles_df.to_pickle(f"his_data/{pair}_{granularity}.pkl")

In [18]:
def create_data(pair, granularity):
    code, json_data = fetch_candles(pair, 4000, granularity)
    if code != 200:
        print(pair, "Error")
        return
    data_f = get_candles_df(json_data)
    print(f"{data_f} loaded {data_f.shape[0]} candles from {data_f.time.min()} to {data_f.time.max()}")
    save_file(data_f, pair, granularity)

In [19]:
for p1 in our_curr:
    for p2 in our_curr:
        pair = f"{p1}_{p2}"
        if pair in ins_df.name.unique():
            create_data(pair, "H1")

                                time  volume    bid_o    bid_h    bid_l   
0     2022-09-14T22:00:00.000000000Z    1327  0.99787  0.99841  0.99784  \
1     2022-09-14T23:00:00.000000000Z    1184  0.99816  0.99832  0.99786   
2     2022-09-15T00:00:00.000000000Z    2582  0.99812  0.99814  0.99710   
3     2022-09-15T01:00:00.000000000Z    3150  0.99747  0.99815  0.99698   
4     2022-09-15T02:00:00.000000000Z    2055  0.99808  0.99819  0.99685   
...                              ...     ...      ...      ...      ...   
3994  2023-05-08T09:00:00.000000000Z    3674  1.10353  1.10386  1.10220   
3995  2023-05-08T10:00:00.000000000Z    2693  1.10353  1.10495  1.10346   
3996  2023-05-08T11:00:00.000000000Z    3591  1.10474  1.10508  1.10294   
3997  2023-05-08T12:00:00.000000000Z    3915  1.10344  1.10404  1.10313   
3998  2023-05-08T13:00:00.000000000Z    4945  1.10365  1.10437  1.10326   

        bid_c    mid_o    mid_h    mid_l    mid_c    ask_o    ask_h    ask_l   
0     0.99815  0.99